In [1]:
import cv2
import numpy as np

def detect_and_match_keypoints(img1_gray, img2_gray):
    sift = cv2.SIFT_create()
    keypoints1, descriptors1 = sift.detectAndCompute(img1_gray, None)
    keypoints2, descriptors2 = sift.detectAndCompute(img2_gray, None)
    index_params = dict(algorithm=1, trees=5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(descriptors1, descriptors2, k=2)
    good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]
    points1 = np.float32([keypoints1[m.queryIdx].pt for m in good_matches])
    points2 = np.float32([keypoints2[m.trainIdx].pt for m in good_matches])
    return points1, points2

def compute_homography(points1, points2):
    H, _ = cv2.findHomography(points1, points2, cv2.RANSAC)
    return H

def warp_image_with_blending(img1, img2, H):
    h1, w1 = img1.shape[:2]
    h2, w2 = img2.shape[:2]
    corners_img1 = np.float32([[0, 0], [0, h1], [w1, h1], [w1, 0]]).reshape(-1, 1, 2)
    warped_corners = cv2.perspectiveTransform(corners_img1, H)
    corners_img2 = np.float32([[0, 0], [0, h2], [w2, h2], [w2, 0]]).reshape(-1, 1, 2)
    all_corners = np.concatenate((warped_corners, corners_img2), axis=0)
    [x_min, y_min] = np.int32(all_corners.min(axis=0).ravel() - 0.5)
    [x_max, y_max] = np.int32(all_corners.max(axis=0).ravel() + 0.5)
    translation_dist = [-x_min, -y_min]
    translation_matrix = np.array([[1, 0, translation_dist[0]], [0, 1, translation_dist[1]], [0, 0, 1]])
    output_img = cv2.warpPerspective(img1, translation_matrix @ H, (x_max - x_min, y_max - y_min))
    output_img[translation_dist[1]:h2 + translation_dist[1], translation_dist[0]:w2 + translation_dist[0]] = img2

    mask1 = np.zeros((h2, w2), dtype=np.uint8)
    mask1[:, :] = 255
    mask1_warped = cv2.warpPerspective(mask1, translation_matrix @ H, (x_max - x_min, y_max - y_min))

    dist_transform_img1 = cv2.distanceTransform(mask1_warped, cv2.DIST_L2, 5)
    dist_transform_img2 = cv2.distanceTransform(255 - mask1_warped, cv2.DIST_L2, 5)
    blend_mask1 = dist_transform_img1 / (dist_transform_img1 + dist_transform_img2 + 1e-10)
    blend_mask2 = dist_transform_img2 / (dist_transform_img1 + dist_transform_img2 + 1e-10)
    blended_output = np.zeros_like(output_img)
    for c in range(3):
        blended_output[:, :, c] = (output_img[:, :, c] * blend_mask1 + img2[:, :, c] * blend_mask2).astype(np.uint8)
    return blended_output

def stitch_multiple_images(images):
    reference_idx = len(images) // 2
    panorama = images[reference_idx]
    for i in range(reference_idx - 1, -1, -1):
        img1_gray = cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY)
        img2_gray = cv2.cvtColor(panorama, cv2.COLOR_BGR2GRAY)
        points1, points2 = detect_and_match_keypoints(img1_gray, img2_gray)
        H = compute_homography(points1, points2)
        panorama = warp_image_with_blending(images[i], panorama, H)
    
    for i in range(reference_idx + 1, len(images)):
        img1_gray = cv2.cvtColor(images[i], cv2.COLOR_BGR2GRAY)
        img2_gray = cv2.cvtColor(panorama, cv2.COLOR_BGR2GRAY)
        points1, points2 = detect_and_match_keypoints(img1_gray, img2_gray)
        H = compute_homography(points1, points2)
        panorama = warp_image_with_blending(images[i], panorama, H)
    return panorama

# Load images in color
image_files = ['STA_0031.jpg', 'STB_0032.jpg', 'STC_0033.jpg', 'STD_0034.jpg', 'STE_0035.jpg', 'STF_0036.jpg']
images = [cv2.imread(f) for f in image_files]
images = [img for img in images if img is not None]

# Stitch the images into a panorama
stitched_panorama = stitch_multiple_images(images)

# Show and save the result
cv2.imshow("Stitched Panorama", stitched_panorama)
cv2.imwrite("stitched_panorama.jpg", stitched_panorama)
cv2.waitKey(0)
cv2.destroyAllWindows()


IndexError: list index out of range